In [1]:

# Funktioner som används av SKB.ipynb
 
import os
import arcpy
arcpy.CheckOutExtension('3D')


def create_SKB_XYmZ(txt_file, gdb_name, fc_name, sr):

    fc_attributes = [['X', 'DOUBLE'],
                     ['Y', 'DOUBLE'],
                     ['Z', 'FLOAT'],
                     ['dZ', 'FLOAT'],
                     ['mZ', 'FLOAT']]

    fc_all_fields = ['X', 'Y', 'Z', 'dZ', 'mZ', 'SHAPE@XY', 'SHAPE@Z']

    txt_file_fields = ['X', 'Y', 'Z', 'dZ']

    arcpy.CreateFeatureclass_management(out_path=gdb_name, out_name=fc_name,
                                        geometry_type='POINT', has_z='ENABLED',
                                        spatial_reference=sr)

    for attribute in fc_attributes:
        arcpy.AddField_management(in_table=os.path.join(gdb_name, fc_name),
                                  field_name=attribute[0], field_type=attribute[1])

    # Read txt file, insert line by line in feature class
    with open(txt_file, 'r') as srcFile:
        fc = os.path.join(gdb_name, fc_name)
        with arcpy.da.InsertCursor(fc, fc_all_fields) as i_cursor:
            for fileLine in srcFile:
                lSplit = fileLine.split(' ')
                X = float(lSplit[0])
                Y = float(lSplit[1])
                Z = float(lSplit[2])
                dZ = float(lSplit[3])
                mZ = Z - dZ
                shapeXYZ = [(X, Y), mZ]
                row = [X, Y, Z, dZ, mZ] + shapeXYZ
                i_cursor.insertRow(row)


def create_SKB_XYZ(gdb, fc, fc_template, sr):
    # function to create fc_XYZ based on delivery template
    # only execute this if the fc does not already exist
    fc_full_path = os.path.join(gdb, fc)
    if arcpy.Exists(fc_full_path):
        print(fc+' already exists. Not created')
    else:
        arcpy.CreateFeatureclass_management(out_path=gdb, out_name=fc,
                                            geometry_type='POINT', template=fc_template,
                                            has_z='SAME_AS_TEMPLATE', spatial_reference=sr)


def dist_mZ_wire(gdb_name, fc_name, wires, radius):
    # Compute distance from tree ground point to wire
    # (Tree height can then be subtracted from this value to compute AVST_FAS)
    fc = os.path.join(gdb_name, fc_name)
    arcpy.Near3D_3d(in_features=fc, near_features=wires, search_radius=radius,
                    location="NO_LOCATION", angle="NO_ANGLE", delta="NO_DELTA")
    print('done with Near3D '+fc_name)
    arcpy.DeleteField_management(in_table=fc, drop_field='NEAR_FID')
    arcpy.AlterField_management(in_table=fc, field='NEAR_DIST',
                                new_field_name='AVSTAND_HORISONTELLT')
    arcpy.AlterField_management(in_table=fc, field='NEAR_DIST3',
                                new_field_name='AVST_MZ_FAS')
    print('done with deletefield '+fc_name)


def LG_name(LG):
    if LG <= 9:
        LG_str = 'LG00'+str(LG)
    elif LG <= 99:
        LG_str = 'LG0'+str(LG)
    else:
        LG_str = 'LG'+str(LG)

    return (LG_str)


def pick_out_akuta_and_traffpunkter(gdb, fc_in, fc_akuta, fc_trfpt, fc_intr_ers):
    # function for selecting
    # 1) akuta trad, and
    # 2) traffpunkter
    # and save in separate feature classes

    arcpy.Delete_management('trfpt_lyr')
    # Output to fc_akuta all trees in fc_in with AVSTAND_FAS <= 1
    fc_in_path = os.path.join(gdb, fc_in)
    arcpy.FeatureClassToFeatureClass_conversion(
        fc_in_path, gdb, fc_akuta, '"AVSTAND_FAS" <= 1', '#', '#')

    # Output to fc_trfpt all trees in fc_in with AVSTAND_FAS > 1
    arcpy.FeatureClassToFeatureClass_conversion(
        fc_in_path, gdb, fc_trfpt, '"AVSTAND_FAS" > 1', '#', '#')
    # Then select and delete trees from fc_trfpt that intersect intrangsersatt mark
    fc_trfpt_path = os.path.join(gdb, fc_trfpt)
    arcpy.MakeFeatureLayer_management(fc_trfpt_path, 'trfpt_lyr')
    arcpy.SelectLayerByLocation_management(
        'trfpt_lyr', 'INTERSECT', fc_intr_ers, '#', 'NEW_SELECTION')
    arcpy.DeleteFeatures_management('trfpt_lyr')
    arcpy.Delete_management('trfpt_lyr')


def pick_out_akuta(gdb, fc_in, fc_akuta):
    # Function for exporting akuta trad (avst_fas <= 1)
    fc_in_path = os.path.join(gdb, fc_in)
    # Export trees with avst_fas <= 1 to fc_akuta
    arcpy.FeatureClassToFeatureClass_conversion(
        fc_in_path, gdb, fc_akuta, '"AVSTAND_FAS" <= 1', '#', '#')


def pick_out_traffpunkter(gdb, fc_in, fc_trfpt, fc_intr_ers):
    # function for selection traffpunkter and save in separate feature class
    arcpy.Delete_management('trfpt_lyr')

    fc_in_path = os.path.join(gdb, fc_in)

    # Output to fc_trfpt all trees in fc_in with AVSTAND_FAS > 1
    arcpy.FeatureClassToFeatureClass_conversion(
        fc_in_path, gdb, fc_trfpt, '"AVSTAND_FAS" > 1', '#', '#')
    # Then select and delete trees from fc_trfpt that intersect intrangsersatt mark
    fc_trfpt_path = os.path.join(gdb, fc_trfpt)
    arcpy.MakeFeatureLayer_management(fc_trfpt_path, 'trfpt_lyr')
    arcpy.SelectLayerByLocation_management(
        'trfpt_lyr', 'INTERSECT', fc_intr_ers, '#', 'NEW_SELECTION')
    arcpy.DeleteFeatures_management('trfpt_lyr')
    arcpy.Delete_management('trfpt_lyr')


def pick_out_ej_akuta(gdb, fc_in, fc_ej_akuta):
    # function for selection traffpunkter and save in separate feature class
    fc_in_path = os.path.join(gdb, fc_in)
    # Export trees with avst_fas > 1 to fc_ej_akuta
    arcpy.FeatureClassToFeatureClass_conversion(
        fc_in_path, gdb, fc_ej_akuta, '"AVSTAND_FAS" > 1', '#', '#')


def populate_SKB_XYZ(input_fc, output_fc, LG, atgardsar, leverantor, ins_met, matosak_p, matosak_h):
    # reads the LG's fc_XYZmZ and writes to the LG'x fc_XYZ"

    LG_code_dict = {1: 10, 2: 20, 3: 30, 4: 40, 5: 50, 6: 60, 7: 70, 8: 80, 9: 90,
                    10: 100, 11: 110, 12: 120, 13: 130, 14: 140, 15: 150, 16: 160, 17: 170, 18: 180, 19: 190,
                    20: 200, 21: 210, 22: 220, 23: 230, 24: 240, 25: 250, 26: 260, 27: 270, 28: 280, 29: 290,
                    30: 300, 31: 310, 32: 320, 33: 330, 34: 340, 35: 350, 36: 360, 37: 370, 38: 380, 39: 390,
                    40: 400, 41: 410, 42: 420, 43: 430, 44: 440, 45: 450, 46: 460, 47: 470, 48: 480, 49: 490,
                    50: 500, 51: 510, 52: 520, 53: 530, 54: 540, 55: 550, 56: 560, 57: 570, 58: 580, 59: 590,
                    60: 600, 61: 610, 62: 620, 63: 630, 64: 640, 65: 650, 66: 660, 67: 670, 68: 680, 69: 690,
                    70: 700, 71: 710, 72: 720, 73: 730, 74: 740, 75: 750, 76: 760, 77: 770, 78: 780, 79: 790,
                    80: 800, 81: 810, 82: 820, 83: 830, 84: 840, 85: 850, 86: 860, 87: 870, 88: 880, 89: 890,
                    90: 900, 91: 910, 92: 920, 93: 930, 94: 940, 95: 950, 96: 951, 97: 952, 98: 953,
                    100: 960, 101: 970, 102: 980, 103: 990, 104: 1010, 105: 1020, 106: 1030, 107: 1040, 108: 1050, 109: 1060,
                    110: 1070, 111: 1080, 112: 1090, 113: 1100, 114: 1110, 115: 1120, 116: 1130, 117: 1140, 118: 1150, 119: 1160,
                    120: 1170, 121: 1180, 122: 1190, 123: 1200, 124: 1210, 125: 1220, 126: 1230, 127: 1240, 128: 1250, 129: 1260,
                    130: 1270, 131: 1280, 132: 1290, 133: 1300, 134: 1310, 135: 1320, 136: 1330, 137: 1340, 138: 1350, 139: 1360,
                    140: 1370, 141: 1380, 142: 1390, 143: 1400,
                    410: 1410, 500: 1420, 550: 1430, 555: 1435, 557: 1440, 661: 1450,
                    700: 1460, 701: 1470, 710: 1480, 720: 1490, 720: 1500, 730: 1510, 740: 1520, 750: 1530, 770: 1540}

    in_fields = ['SHAPE@XY', 'Z', 'dZ', 'AVSTAND_HORISONTELLT', 'AVST_MZ_FAS']

    out_fields = ['SHAPE@XY', 'SHAPE@Z', 'DELTA_HOJD', 'AVSTAND_FAS', 'MAX_TILLVAXT',
                  'AVSTAND_HORISONTELLT', 'LEDNINGSGATA', 'ATGARDSAR', 'LEVERANTOR',
                  'INSAMLINGSMETOD', 'MATOSAKERHET_PLAN', 'MATOSAKERHET_HOJD']

    LG_code = LG_code_dict[LG]

    with arcpy.da.SearchCursor(input_fc, in_fields) as s_cursor:
        with arcpy.da.InsertCursor(output_fc, out_fields) as i_cursor:
            for row in s_cursor:
                avst_fas = (row[4] - row[2])
                max_tillv = avst_fas - 1
                delta_hojd = row[2]
                avst_hori = row[3]
                z = row[1]  # - row[2]
                out_row = [row[0], z, delta_hojd, avst_fas, max_tillv, avst_hori, LG_code, atgardsar,
                           leverantor, ins_met, matosak_p, matosak_h]

                i_cursor.insertRow(out_row)


def remove_duplicates(gdb, fc_in, fc_out, xy_tol, z_tol):
    # Removes duplicates among akuta and traffpunkter, based on location.
    # Assumes that all instances in a "duplicate group" have identical
    # AVST_FAS etc (true if mZ was calculated for whole LG (all batches)
    # together, on all wires in the LG)
    input_fc = os.path.join(gdb, fc_in)
    no_dupl_fc = os.path.join(gdb, fc_out)

    arcpy.Copy_management(input_fc, no_dupl_fc)

    arcpy.DeleteIdentical_management(no_dupl_fc, ['Shape'], xy_tol, z_tol)


def compute_horizontal_dist(fc_in, wires, radius):
    # NEAR_DIST computed by Near3D, and then used as AvstHori in populate_SKB_XYZ,
    # does not seem to be the perpendicular distance to the wire, but rather the
    # distance to the nearest point on the wire (due to sag, I guess)
    # So: Use NEAR analysis in 2D to create a new NEAR column, then use that one as AvstHori
    arcpy.Near_analysis(in_features=fc_in, near_features=wires, search_radius=radius,
                        location="NO_LOCATION", angle="NO_ANGLE", method="PLANAR")


def update_z_coordinate(fd, fc):
    """New in 2023

    Updating z-coordinate from beeing treetop height to
    ground point z
    """

    arcpy.env.workspace = os.path.join(fd, fc)
    field = ['SHAPE@Z']
    cursor = arcpy.UpdateCursor(fc, field)

    s_field = ['DELTA_HOJD']
    s_cursor = arcpy.SearchCursor(fc, s_field)

    with arcpy.da.SearchCursor(fc, s_field) as s_cursor:
        with arcpy.da.UpdateCursor(fc, field) as cursor:
            for s_row, row in zip(s_cursor, cursor):
                row[0] = (row[0] - s_row[0])  # float(s_row[0])
                cursor.updateRow(row)

        # arcpy.AddField_management(fc,
        #                          "z", field_type="FLOAT")
        # geo = ['!Shape!.firstpoint.z']
        # arcpy.CalculateField_management(fc, "z", geo[0])


def update_horizontal_dist(fc_in):
    # put the horizontal distance calculated in compute_horizontal_dist
    # in the AvstHori field, and then remove the NEAR field
    near_field = 'NEAR_DIST'
    near_fid_field = 'NEAR_FID'
    avst_hori_field = 'AVSTAND_HORISONTELLT'
    update_fields = [avst_hori_field, near_field]

    with arcpy.da.UpdateCursor(fc_in, update_fields) as u_cursor:
        for row in u_cursor:
            row[0] = row[1]
            u_cursor.updateRow(row)

    arcpy.DeleteField_management(in_table=fc_in, drop_field=near_field)
    arcpy.DeleteField_management(in_table=fc_in, drop_field=near_fid_field)


In [2]:
from pathlib import Path
import shutil
import fileinput
import os
import glob


def create_dir(*dirpaths):
    import os
    for dirpath in dirpaths:
        if not os.path.exists(dirpath):
            os.makedirs(dirpath)
            print(f"{dirpath}  created.")
        else:
            print(f"{dirpath}  already exists. Not overwritten.")


def copy_file(src, dest):
    try:
        shutil.copy(src, dest)
    # e.g. src and dest are the same file
    except shutil.Error as e:
        print(f"Error: {e}")
    # e.g. src or dest doesn't exist
    except IOError as e:
        print(f"Error: e.strerror")


def merge_shapefiles(infiles):
    import geopandas as gpd
    import pandas as pd
    indata = [gpd.read_file(infile) for infile in infiles]
    indata_combined = gpd.GeoDataFrame(pd.concat(indata, ignore_index=True), crs=indata[0].crs)
    return(indata_combined)


def merge_blocks(src_dir, search_pattern, dst_file):
    blocks = glob.glob(os.path.join(src_dir, search_pattern))
    with open(dst_file, "w") as fh:
        input_lines = fileinput.input(blocks)
        fh.writelines(input_lines)
        

def replace_line_in_file(infile, newfile, oldLine, newLine):
    with open(infile, 'r') as input_file, open(newfile, 'w') as output_file:
        for line in input_file:
            if line.strip() == oldLine:
                output_file.write(f"{newLine}\n")
            else:
                output_file.write(line)



## Projektsökvägar
Sätter nödvändiga sökvägar som kommer att användas genom hela skriptets gång.

In [13]:
### KARIN LOKALT ###
import os
from pathlib import Path
prj_dir = Path(r"C:\SVK_2024\pythonkörningar")

# Namn på styrfil med information om de ledningar som ska köras:
LGs_info_name = 'veg_kanttrad_241206.txt'
LGs_info_path = os.path.join(r"Q:\Projekt\Data_2024\styrfiler", LGs_info_name)

# Namn på den gdb där data ska läggas. Den måste finnas och ha domäner inlagda. 
# Mall att kopiera finns i GitHub-repot under python\gdb-mallar-och-domänkoder.
gdb_name = 'veg_kantträd_241206.gdb'
gdb = os.path.join(prj_dir, gdb_name)

# uppdragsmappens mapp "ledningar" där alla terrascan-resultat hamnar:
serv_prj_dir = Path(r"Q:\Projekt\Analys_2024\ledningar")

# domäner som används:
cvd_LEDNINGSGATA = 'cvd_LEDNINGSGATA.txt'
cvd_LEDNINGSGATA_path = os.path.join(r"C:\Users\SEKNNO\OneDrive - Sweco AB\SVK_2024\30047770-SVK-Analys-av-laserdata-extern\python\gdb-mallar-och-domänkoder", 
                                     cvd_LEDNINGSGATA)
skanningsdatum_path = r"Q:\Projekt\Data_2024\skanningsdatum\skanningsdatum_241202.txt"


#all_merged_name = "2024_11_11"
#version = "2024_11_11"

# Merge SKB text files for all blocks of a powerline
Kör blockskript i respektive servermapp för att ha den absolut senaste uppdaterade.

In [13]:
import os
import sys
import glob
import fileinput
from pathlib import Path
#sys.path.insert(1, r"C:\Users\semnve\Documents\Visual Studio Code\30047770-SVK-Analys-av-laserdata")
#import SKB
#import SVK
import pandas as pd

def combine_blocks(row):
    LG = row[0]
    line = row[1]
    line_dir = serv_prj_dir / LG / f"line_{line}"
    
    print(f"Doing {LG}_{line}")
    block_dir = os.path.join(line_dir, "kantträd", "block")
    combined_blocks_path = os.path.join(line_dir, "kantträd", "SKB_raw.txt")
    
    # Merge files for all blocks into one
    #SVK.merge_blocks(block_dir, "*.txt", combined_blocks_path)
    merge_blocks(block_dir, "*.txt", combined_blocks_path)

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
print(LGs_info)
LGs_info.apply(combine_blocks, axis=1)

       LG  line    Littera  ...                         Namn     Status Längd km
0   LG001     1  UL24 S5-6  ...                Ritsem-Ofoten  Befintlig     44,1
1   LG002     1   UL1 S1-3  ...     Porjusberget - Grundfors  Befintlig      1,8
2   LG002     2    UL24 S1  ...           Harsprånget-Porjus  Befintlig      0,1
3   LG002     3  UL24 S2-3  ...          Porjusberget-Vietas  Befintlig     95,4
4   LG003     1   UL1 S1-3  ...     Porjusberget - Grundfors  Befintlig     19,7
5   LG003     2    UL24 S1  ...           Harsprånget-Porjus  Befintlig      8,5
6   LG003     3       UL27  ...            Harsprånget-Ligga  Befintlig     10,5
7   LG003     4       UL29  ...            Harsprånget-Letsi  Befintlig     42,9
8   LG003     5     UL6 S1  ...               Ligga-Messaure  Befintlig     21,4
9   LG004     1    UL24 S4  ...                Vietas-Ritsem  Befintlig     48,2
10  LG005     1   UL1 S1-3  ...     Porjusberget - Grundfors  Befintlig     28,9
11  LG005     2       UL25  

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
dtype: object

## Kopiera skb_raw.txt
Denna kopierar över alla skb_raw.txt till den temporära mappen på skrivbordet

In [9]:
import os
import filecmp
import shutil
import pandas as pd
df = pd.DataFrame()
to_dir = os.path.join(prj_dir, "ledningar")
from_dir = serv_prj_dir

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
from_dir_SKB = []
for i, j in zip(LGs_info["LG"], LGs_info["line"]):
    from_dir_SKB.append(f'{i}\\line_{j}\\Kantträd')
to_dir_SKB = []
for i, j in zip(LGs_info["LG"], LGs_info["line"]):
    to_dir_SKB.append(f'{i}\\line_{j}\\SKB')

outlist_to_dir = []
for i in to_dir_SKB:
    outlist_to_dir.append((f"{to_dir}\{i}\SKB_raw.txt").replace('\\\\', '\\'))

outlist_from_dir = []
for i in from_dir_SKB:
    outlist_from_dir.append(
        (f"{from_dir}\{i}\SKB_raw.txt").replace('\\\\', '\\'))

for i, j in zip(outlist_from_dir, outlist_to_dir):
    if os.path.exists(j.replace('\\\\', '\\')):
        shutil.copyfile(i.replace('\\\\', '\\'), j.replace('\\\\', '\\'))
    else:
        folder1 = j.split("\\")[-4]
        folder2 = j.split("\\")[-3]
        folder3 = j.split("\\")[-2]
        os.makedirs(
            os.path.join(
                to_dir,
                folder1,
                folder2,
                folder3,
                #"SKB"
            )
        )
        shutil.copyfile(i.replace('\\\\', '\\'), j.replace('\\\\', '\\'))

for i, j in zip(outlist_from_dir, outlist_to_dir):
    print(filecmp.cmp(i, j))


True
True


# Calculate distances to wire for trees

In [5]:
import sys
import os, time, arcpy
from time import gmtime, strftime
from pathlib import Path
import pandas as pd
#import SKB

# Featuredataset för kantträdspunkter innehållande X, Y och markens Z-värde under trädet:
SKB_XYmZ_location = os.path.join(gdb, 'SKB_XYmZ')

# Featuredataset för kantträdspunkter innehållande X, Y och Z-värde för trädkronans topp:
SKB_XYZ_location = os.path.join(gdb, 'SKB_XYZ')


# Featuredataset för faslinor, som i ett tidigare steg lästs in från DGN-filerna (Wires.ipynb):
#wires_fd_1 = os.path.join(gdb, 'Wires_2041205')
wires_fd_1 = r"C:\SVK_2024\pythonkörningar\Wires_241205.gdb"
# wires_fd_2 = os.path.join(gdb, 'Wires_incomplete') # Troligen en rest från när vi hade dåliga punktmoln som orsakade luckor i faslinorna
wires_suffix = '_fas'
search_radius = 100 # How far from the point should Near3D search for wires
SKB_template = os.path.join(gdb, 'SKB_template') # Detta bör vara en mall för featureklassen med resultatet som ska skapas
atgardsar = 2024 # TODO: BÖR INTE VARA MED LÄNGRE, KOLLA SÅ ATT DET INTE ANVÄNDS LÄNGRE NER I SKRIPTET
leverantor = 'SWECO'
insamlingsmetod = 20
matosakerhet_p = 1000
matosakerhet_h = 1000
arcpy.env.overwriteOutput = True

def sweref99TM():
    wkt = 'PROJCS["SWEREF99_TM",GEOGCS["GCS_SWEREF99",DATUM["D_SWEREF99",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["Meter",1]]'
    spatialref = arcpy.SpatialReference()
    spatialref.loadFromString(wkt)
    return(spatialref)


def compute_SKB_distances(row):
    LG = row[0] # TODO: Hänvisa till kolumnnamn i styrfilen istället för index
    line = row[1] # TODO: Hänvisa till kolumnnamn i styrfilen istället för index
    line_dir = serv_prj_dir / LG / f"line_{line}"
    SKB_dir = line_dir / "kantträd" 
    
    arcpy.env.overwriteOutput = True
    print(f"Starting with {LG}_{line}")
    
    # Namn på featureklasser som ska skapas i featuredataset som definierats i början av skriptet:
    SKB_XYmZ = f"{LG}_{line}_SKB_XYmZ"
    SKB_XYZ = f"{LG}_{line}_SKB_XYZ"
    akuta_trad = f"{LG}_{line}_akuta"
    ej_akuta_trad = f"{LG}_{line}_ej_akuta"
    
    # Sökvägar till featureklasserna:
    SKB_XYmZ_path = os.path.join(SKB_XYmZ_location, SKB_XYmZ)
    SKB_XYZ_path = os.path.join(SKB_XYZ_location, SKB_XYZ)
    akuta_trad_path = os.path.join(SKB_XYZ_location, akuta_trad) # Används ej? Inaktuellt eftersom vi inte längre delar upp kantträd i akuta och ej akuta
    ej_akuta_trad_path = os.path.join(SKB_XYZ_location, ej_akuta_trad) # Används ej? Inaktuellt eftersom vi inte längre delar upp kantträd i akuta och ej akuta

    wires_1 = os.path.join(wires_fd_1, f"{LG}_{line}{wires_suffix}")
    # wires_2 = os.path.join(wires_fd_2, f"{LG}_{line}{wires_suffix}") # REST SOM INTE LÄNGRE SKA VARA MED? DEN FANNS NÄR VI HADE DÅLIGA PUNKTMOLN SOM ORSAKADE LUCKOR I FASLINORNA
    
    if arcpy.Exists(wires_1):
        wires = wires_1
        print(f"{LG}_{line} faslinor i Wires")
    #elif arcpy.Exists(wires_2):
    #    wires = wires_2
    #    print(f"{LG}_{line} faslinor i Wires_incomplete")
    else:
        print(f"Inga faslinor för {LG}_{line}")
        return
    
    # Create an fc for X,Y,Z,dZ,mZ with mZ as Z coordinate
    # Z är trädtoppens Z-värde i RH2000 dvs höjd över havet
    # dZ är skillanden mellan markens Z-värde under trädet och trädkronans Z-värde
    # mZ är markens Z-värde under trädet
    # Output från TerraScan innehåller X, Y, Z och dZ.
    SKB_file_in = os.path.join(SKB_dir, f"SKB_raw.txt")
    #SKB.create_SKB_XYmZ(SKB_file_in, SKB_XYmZ_location, SKB_XYmZ, sr)
    create_SKB_XYmZ(SKB_file_in, SKB_XYmZ_location, SKB_XYmZ, sr)
    print(f'{LG}_{line}_SKB_XYmZ created')

    # Call arcpy function Near3D to calculate distance from mZ to wire, 
    # on SKB_XYmZ
    #SKB.dist_mZ_wire(SKB_XYmZ_location, SKB_XYmZ, wires, search_radius)
    dist_mZ_wire(SKB_XYmZ_location, SKB_XYmZ, wires, search_radius)
    print(f'dist_mZ_wire calculated for {LG}_{line}')
    print(f'{strftime("%H:%M:%S", gmtime())}')

    # Create feature class for SKB where point XYZ is the XYZ for the tree tops
    #SKB.create_SKB_XYZ(SKB_XYZ_location, SKB_XYZ, SKB_template, sr)
    create_SKB_XYZ(SKB_XYZ_location, SKB_XYZ, SKB_template, sr)

    # Fill SKB_XYZ with values from SKB_XYmZ and set Z coordinate to Z (instead of mZ)
    LG_nr = int(LG[-3:])
    #SKB.populate_SKB_XYZ(SKB_XYmZ_path, SKB_XYZ_path, LG_nr, atgardsar, leverantor, insamlingsmetod,
    #                    matosakerhet_p, matosakerhet_h)
    populate_SKB_XYZ(SKB_XYmZ_path, SKB_XYZ_path, LG_nr, atgardsar, leverantor, insamlingsmetod, matosakerhet_p, matosakerhet_h)
    print('SKB_XYZ populated')  

    # NEAR_DIST computed by Near3D, and then used as AvstHori in populate_SKB_XYZ,
    # does not seem to be the perpendicular distance to the wire, but rather the 
    # distance to the nearest point on the wire (due to sag, I guess)
    # So: Use NEAR analysis in 2D to create a new NEAR column, then use that one as AvstHori
    #SKB.compute_horizontal_dist(SKB_XYZ_path, wires, search_radius)
    compute_horizontal_dist(SKB_XYZ_path, wires, search_radius)
    print('Avstand fas computed')
          
    # Now replace old avst_hori with new, correct one from compute_horizontal_dist. 
    # Then remove the NEAR-fields.
    #SKB.update_horizontal_dist(SKB_XYZ_path)
    update_horizontal_dist(SKB_XYZ_path)
    print('Avstand horisontellt computed')
     
    print("Setting ground Z for XYZ points")
    all_fc = os.path.join(SKB_XYZ_path, SKB_XYZ)
    #SKB.update_z_coordinate(SKB_XYZ_location, SKB_XYZ)
    update_z_coordinate(SKB_XYZ_location, SKB_XYZ)

    
          
    print(f"Done with {LG}_{line}")
# Coordinate system
sr = sweref99TM()

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(compute_SKB_distances, axis=1)
    


Starting with LG003_5
LG003_5 faslinor i Wires
LG003_5_SKB_XYmZ created
done with Near3D LG003_5_SKB_XYmZ
done with deletefield LG003_5_SKB_XYmZ
dist_mZ_wire calculated for LG003_5
08:18:51
SKB_XYZ populated
Avstand fas computed
Avstand horisontellt computed
Setting ground Z for XYZ points
Done with LG003_5
Starting with LG004_1
LG004_1 faslinor i Wires
LG004_1_SKB_XYmZ created
done with Near3D LG004_1_SKB_XYmZ
done with deletefield LG004_1_SKB_XYmZ
dist_mZ_wire calculated for LG004_1
08:19:50
SKB_XYZ populated
Avstand fas computed
Avstand horisontellt computed
Setting ground Z for XYZ points
Done with LG004_1
Starting with LG005_1
LG005_1 faslinor i Wires
LG005_1_SKB_XYmZ created
done with Near3D LG005_1_SKB_XYmZ
done with deletefield LG005_1_SKB_XYmZ
dist_mZ_wire calculated for LG005_1
08:20:37
SKB_XYZ populated
Avstand fas computed
Avstand horisontellt computed
Setting ground Z for XYZ points
Done with LG005_1
Starting with LG005_2
LG005_2 faslinor i Wires
LG005_2_SKB_XYmZ created
d

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
dtype: object

## Städa bland fält och fältnamn
## VARFÖR TAS FÄLTEN LEDNINGSGATA OCH INSAMLINGSMETOD BORT? DE LÄGGS SEDAN TILL  IGEN. HAR DE FEL FORMAT I MALLDATABASEN? BÄTTRE ATT ÄNDRA DET I SÅ FALL.


In [ ]:
#import arcpy, os, glob, shutil
#from pathlib import Path
#import pandas as pd
#
#SKB_XYZ_fd = os.path.join(prj_dir, gdb ,"SKB_XYZ")                                                                                            
#
#def alterfield(powerline):    
#    LG = powerline[0]
#    line = powerline[1]
#    
#    fieldnames = ["_SKB_XYZ"]
#    
#    for fieldname in fieldnames:
#        
#        fc_SKB = f"{LG}_{line}{fieldname}"
#        SKB_path = os.path.join(SKB_XYZ_fd, f"{LG}_{line}{fieldname}")
#        print(f"Altering fields for {LG}_{line}{fieldname}")
#        path_joined = os.path.join(SKB_XYZ_fd, f"{LG}_{line}{fieldname}")
#        field_names = [f.name for f in arcpy.ListFields(path_joined)]
#        # Activating workspace
#        arcpy.env.workspace = SKB_path
#        arcpy.env.overwriteOutput = True
#        
#        if "LEDNINGSGATA" in field_names:    
#            arcpy.management.DeleteField(
#                path_joined,
#                "LEDNINGSGATA")
#        if "INSAMLINGSMETOD" in field_names:    
#            arcpy.management.DeleteField(
#                path_joined,
#                "INSAMLINGSMETOD")
#
#LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
#LGs_info.apply(alterfield, axis=1)

Altering fields for LG040_1_SKB_XYZ
Altering fields for LG040_2_SKB_XYZ
Altering fields for LG040_3_SKB_XYZ
Altering fields for LG040_4_SKB_XYZ
Altering fields for LG040_5_SKB_XYZ
Altering fields for LG040_6_SKB_XYZ
Altering fields for LG661_1_SKB_XYZ
Altering fields for LG661_2_SKB_XYZ


0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
dtype: object

# Fyll i Littera, Ursprung, Ursprung_Datum, Registreringsdatum
Borde göras i ett tidigare steg

In [17]:
# Add data from LEDNIGNAR.txt
import os
import pandas as pd
from pathlib import Path
import datetime
from datetime import date
import arcpy

SKB_location = os.path.join(gdb, 'SKB_XYZ')

date_field = 'RGDTM'
#matosakerhet_h_field = 'MATOSAKERHET_HOJD'

urspr = "SWECO"
#ins_met = 20
#matosak_plan = 1000
#matosak_hojd = 1000
lev_dat = str(date.today())

def add_date_and_accuracy(powerline):
    LG = powerline["LG"]
    line = powerline["line"]
    littera = powerline["Littera"]
    #LG_code = cvd_LEDNINGSGATA[LG]
    regdatum = skanningsdatum[f"{LG}_{line}"]
    
    LG_line = f"{LG}_{line}"
    print(f"Doing {LG_line}")

    fc_SKB = f"{LG}_{line}_SKB_XYZ"
    SKB_path = os.path.join(SKB_location, fc_SKB)
    # Activating workspace
    arcpy.env.workspace = SKB_path
    arcpy.env.overwriteOutput = True
    

    
    arcpy.management.CalculateField(fc_SKB, "LITTERA", "".join(("'", littera, "'")))
    arcpy.management.CalculateField(fc_SKB, "Ursprung", "".join(("'", urspr, "'")))
    arcpy.management.CalculateField(fc_SKB, "Ursprung_Datum", "".join(("'", lev_dat, "'")))
    arcpy.management.CalculateField(fc_SKB, "RGDTM", "".join(("'", regdatum, "'")))
    #sr = arcpy.SpatialReference("SWEREF99_TM")
    #arcpy.DefineProjection_management(fc_SKB, sr)
    # Assigning domains to fields
    arcpy.management.AssignDomainToField(
        fc_SKB, "LITTERA", "cvd_LITTERA_LEDNING")
    
    print("Klart")

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
df_skanningsdatum = pd.read_csv(skanningsdatum_path, sep="\t", header=0)
skanningsdatum = {f"{row['LG']}_{row['line']}": row['skanningsdatum'] for _, row in df_skanningsdatum.iterrows()}

#df_cvd_LEDNINGSGATA = pd.read_csv(cvd_LEDNINGSGATA_path, sep="\t", header=0)
#df_cvd_DATE = pd.read_csv(cvd_DATE_path, sep="\t", header=0)
#cvd_LEDNINGSGATA = {df_cvd_LEDNINGSGATA.LG[i]: df_cvd_LEDNINGSGATA.Code[i] for i in range(
#    len(df_cvd_LEDNINGSGATA))}
#cvd_DATE = {df_cvd_DATE.LG[i].join(
#    "_00" + str(df_cvd_DATE.nr[i])): df_cvd_DATE.Datum[i] for i in range(len(df_cvd_DATE))}

LGs_info.apply(add_date_and_accuracy, axis=1)


Doing LG001_1
Klart
Doing LG002_1
Klart
Doing LG002_2
Klart
Doing LG002_3
Klart
Doing LG003_1
Klart
Doing LG003_2
Klart
Doing LG003_3
Klart
Doing LG003_4
Klart
Doing LG003_5
Klart
Doing LG004_1
Klart
Doing LG005_1
Klart
Doing LG005_2
Klart
Doing LG005_3
Klart
Doing LG005_4
Klart
Doing LG014_1
Klart
Doing LG014_2
Klart
Doing LG014_3
Klart
Doing LG014_4
Klart
Doing LG018_1
Klart
Doing LG018_2
Klart
Doing LG018_3
Klart
Doing LG046_1
Klart
Doing LG046_2
Klart
Doing LG046_4
Klart
Doing LG046_5
Klart


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
dtype: object

## Ändra data i fält:
Avrunda till en decimal, fixa rätt fältnamn och alias

In [11]:
#import arcpy, os, glob, shutil
#from pathlib import Path
#import pandas as pd
#
#def alterfield(powerline):    
#    LG = powerline[0]
#    line = powerline[1]
# 
#    fieldnames = ["_SKB_XYZ"]
#    
#    for fieldname in fieldnames:
#        fc_SKB = f"{LG}_{line}{fieldname}"
#        SKB_path = os.path.join(SKB_location, f"{LG}_{line}{fieldname}")
#        # Activating workspace
#        arcpy.env.workspace = SKB_path
#        arcpy.env.overwriteOutput = True
#        
#        if arcpy.Exists(fc_SKB):
#            SKB_XYZ_fc = f"{LG}_{line}{fieldname}"
#            print(f"Altering fields for {LG}_{line}{fieldname}")
#            path_joined = os.path.join(SKB_XYZ_fd, f'{LG}_{line}{fieldname}')
#            field_names = [f.name for f in arcpy.ListFields(path_joined)]
#            if "AVSTAND_HORISONTELLT" in field_names: 
#                arcpy.management.AlterField(
#                    path_joined,
#                    "AVSTAND_HORISONTELLT",
#                    "AVSTAND_HORISONTELLT",
#                    "AVSTAND_HORISONTELLT")
#
#                arcpy.env.workspace = SKB_XYZ_fd
#                field = ['AVSTAND_HORISONTELLT']
#                cursor = arcpy.UpdateCursor(SKB_XYZ_fc,field)
#
#                with arcpy.da.UpdateCursor(SKB_XYZ_fc,field) as cursor:
#                    for row in cursor:
#                        row[0] = float("{:.1f}".format(round(row[0], 1)))
#                        cursor.updateRow(row)
#
#            if "AVSTAND_FAS" in field_names:    
#                arcpy.management.AlterField(
#                    path_joined,
#                    "AVSTAND_FAS",
#                    "AVSTAND_FAS",
#                    "AVSTAND_FAS")
#
#                arcpy.env.workspace = SKB_XYZ_fd
#                field = ['AVSTAND_FAS']
#                cursor = arcpy.UpdateCursor(SKB_XYZ_fc,field)
#
#                with arcpy.da.UpdateCursor(SKB_XYZ_fc,field) as cursor:
#                    for row in cursor:
#                        row[0] = float("{:.1f}".format(round(row[0], 1)))
#                        cursor.updateRow(row)
#
#            if "DELTA_HOJD" in field_names:    
#                arcpy.management.AlterField(
#                    path_joined,
#                    "DELTA_HOJD",
#                    "DELTA_HOJD",
#                    "DELTA_HOJD")
#
#                arcpy.env.workspace = SKB_XYZ_fd
#                field = ['DELTA_HOJD']
#                cursor = arcpy.UpdateCursor(SKB_XYZ_fc,field)
#
#                with arcpy.da.UpdateCursor(SKB_XYZ_fc,field) as cursor:
#                    for row in cursor:
#                        row[0] = float("{:.1f}".format(round(row[0], 1)))
#                        cursor.updateRow(row)
#
#            if "RGDTM" in field_names: 
#                arcpy.management.AlterField(
#                    path_joined,
#                    "RGDTM",
#                    "RGDTM",
#                    "RGDTM")
#
#            if "MATOSAKERHET_PLAN" in field_names: 
#                arcpy.management.AlterField(
#                    path_joined,
#                    "MATOSAKERHET_PLAN",
#                    "Matosakerhet_Plan",
#                    "Matosakerhet_Plan")
#
#            if "MATOSAKERHET_HOJD" in field_names: 
#                arcpy.management.AlterField(
#                    path_joined,
#                    "MATOSAKERHET_HOJD",
#                    "Matosakerhet_Hojd",
#                    "Matosakerhet_Hojd")
#
#            if "LEDNINGSGATA" in field_names: 
#                arcpy.management.AlterField(
#                    path_joined,
#                    "LEDNINGSGATA",
#                    "LEDNINGSGATA",
#                    "LEDNINGSGATA")
#
#
#    else:
#        print(f"No RBX for {LG}_{line}")
#    
#LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
#LGs_info.apply(alterfield, axis=1)

Altering fields for LG040_1_SKB_XYZ
No RBX for LG040_1
Altering fields for LG040_2_SKB_XYZ
No RBX for LG040_2
Altering fields for LG040_3_SKB_XYZ
No RBX for LG040_3
Altering fields for LG040_4_SKB_XYZ
No RBX for LG040_4
Altering fields for LG040_5_SKB_XYZ
No RBX for LG040_5
Altering fields for LG040_6_SKB_XYZ
No RBX for LG040_6
Altering fields for LG661_1_SKB_XYZ
No RBX for LG661_1
Altering fields for LG661_2_SKB_XYZ
No RBX for LG661_2


0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
dtype: object

# Slå ihop kantträd för alla ledningar till en featureclass

In [12]:
## Merge all to one
#import datetime, arcpy, os
#from pathlib import Path
#import pandas as pd
#
#all_location = os.path.join(gdb, 'SKB_XYZ')
#all_merged_location = os.path.join(gdb, 'all')
#all_merged_path = os.path.join(all_merged_location, all_merged_name)
#
#def to_all_list(powerline):
#    LG = powerline[0]
#    line = powerline[1]
#    
#    print(f"Adding {LG}_{line} to list")
#    
#    all_path = os.path.join(all_location, f"{LG}_{line}_SKB_XYZ")
#
#    all_list.append(all_path)
#    
#    print(f"Done adding {LG}_{line} to list")
#
#    
#all_list = []
#
#LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
#LGs_info.apply(to_all_list, axis=1)
#
#print("Doing merge")
#arcpy.Merge_management(all_list, all_merged_path)
#

Adding LG040_1 to list
Done adding LG040_1 to list
Adding LG040_2 to list
Done adding LG040_2 to list
Adding LG040_3 to list
Done adding LG040_3 to list
Adding LG040_4 to list
Done adding LG040_4 to list
Adding LG040_5 to list
Done adding LG040_5 to list
Adding LG040_6 to list
Done adding LG040_6 to list
Adding LG661_1 to list
Done adding LG661_1 to list
Adding LG661_2 to list
Done adding LG661_2 to list
Doing merge


<Result 'C:\\Users\\semnve\\Desktop\\RBX_och_SKB\\SKB_2023_11_01\\SKB_2023_11_01.gdb\\all\\all_2023_11_01'>

# Klipp träffpunkter
Se till att använda aktuella data i fc_ledningsgator och fc_stationsområden

In [19]:
import arcpy
import os
import pandas as pd

fc_ledningsgator = r"Q:\Projekt\Data_2024\Underlag_SVK\Bestallningsunderlag_2024.gdb\GNG_LEDNINGSGATA" #os.path.join(gdb, "Gator2021")
fc_stationsomraden = r"Q:\Projekt\Data_2024\Underlag_SVK\Bestallningsunderlag_2024.gdb\GNG_STATIONSOMRADE" #os.path.join(gdb, "STATIONSOMRADE")

arcpy.env.overwriteOutput = True

##fd_ej_akuta = os.path.join(gdb, "ej_akuta")
#fd_all = os.path.join(gdb, "all")
#fd_traffpunkter = os.path.join(gdb, "traffpunkter")

#fc_all = os.path.join(fd_all, f"all_{version}")
#fc_traffpunkter = os.path.join(fd_traffpunkter, f"traffpunkter_{version}")
tmp_layer_traffpunkter = os.path.join(gdb, "tmp_layer_traffpunkter")
fc_traffpunkter = os.path.join(gdb, 'kantträd_klippta')
fc_all = os.path.join("kantträd_tmp")
arcpy.management.Delete(tmp_layer_traffpunkter)
arcpy.management.MakeFeatureLayer(fc_all, tmp_layer_traffpunkter)


# Steg 1
# Inom fastbredd och stationsområden: ta bort alla inom 7 m horisontellt avstånd från fas
#
# 1.1 Välj alla inom fastbredd
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = fc_ledningsgator,
                                       selection_type = "NEW_SELECTION")
# 1.2 Lägg till alla inom stationsområden
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = fc_stationsomraden,
                                       selection_type = "ADD_TO_SELECTION")
# 1.3 Av de valda, behåll punkter med avst_hori < 7 m
arcpy.management.SelectLayerByAttribute(in_layer_or_view = tmp_layer_traffpunkter,
                                        selection_type = "SUBSET_SELECTION", 
                                        where_clause = '"AVSTAND_HORISONTELLT" < 7')
# 1.4 Radera valda
arcpy.management.DeleteFeatures(tmp_layer_traffpunkter)

# Steg 2
# Inom fastbredd och stationsområden: ta bort alla med trädhöjd <= 4.5 m och avst_fas >= 5 m
#
# 2.1 Välj alla inom fastbredd
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = fc_ledningsgator,
                                       selection_type = "NEW_SELECTION")
# 2.2 Lägg till alla inom stationsområden
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = fc_stationsomraden,
                                       selection_type = "ADD_TO_SELECTION")
# 2.3 Av de valda, behåll punkter med trädhöjd <= 4.5 OCH avst_fas >= 5
arcpy.management.SelectLayerByAttribute(in_layer_or_view = tmp_layer_traffpunkter,
                                       selection_type = "SUBSET_SELECTION",
                                       where_clause = '"DELTA_HOJD" <= 4.5 AND "AVSTAND_FAS" >= 5')
# 2.4 Radera valda
arcpy.management.DeleteFeatures(tmp_layer_traffpunkter)

# Kopiera ej_akuta till traffpunkter
arcpy.management.CopyFeatures(tmp_layer_traffpunkter, fc_traffpunkter)

# Ta bort det temporära lagret
arcpy.management.Delete(tmp_layer_traffpunkter)


<Result 'true'>

# Rensning onödig information
Rensar rådata från onödig information

Första cellen gör rensningen på de ej hopslagna featureklasserna dvs varje ledninge för sig

Andra cellen gör rensningen på featureklassen med alla ledningar hopslagna (hopslagningen görs under rubrik "Slå ihop kantträd för alla ledningar till en featureklass" ovan)

TODO: gör rensningen på enskilda ledningar innan hopslagningen görs. Kontrollera om rensningen även kan göras innan klippningen, eller om klippningen är beroende av några av de fält som här rensas bort

In [14]:
import arcpy, os, glob, shutil
from pathlib import Path
import pandas as pd


SKB_XYZ_fd = os.path.join(prj_dir, gdb, "SKB_XYZ")                                                                                            

def alterfield(powerline):    
    LG = powerline[0]
    line = powerline[1]
    
    fieldnames = ["_SKB_XYZ"]
    
    for fieldname in fieldnames:
        fc_SKB = f"{LG}_{line}{fieldname}"
        SKB_path = os.path.join(SKB_XYZ_fd, fc_SKB)
        # Activating workspace
        arcpy.env.workspace = SKB_path
        arcpy.env.overwriteOutput = True
        
        
        if arcpy.Exists(fc_SKB):
            SKB_XYZ_fc = f"{LG}_{line}{fieldname}"
            print(f"Altering fields for {LG}_{line}{fieldname}")
            path_joined = os.path.join(SKB_XYZ_fd, f'{LG}_{line}{fieldname}')
            field_names = [f.name for f in arcpy.ListFields(path_joined)]

            if "GIS_ID" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "GIS_ID")
            if "MAX_TILLVAXT" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "MAX_TILLVAXT")
            if "created_user" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "created_user")
            if "created_date" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "created_date")
            if "last_edited_user" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "last_edited_user")
            if "last_edited_date" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "last_edited_date")
            if "ATGARDSAR" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "ATGARDSAR")
            if "LEVERANTOR" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "LEVERANTOR")
            if "LEVERANTORS_DATUM" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "LEVERANTORS_DATUM")
    else:
        print(f"No RBX for {LG}_{line}")
    
LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(alterfield, axis=1)

Altering fields for LG040_1_SKB_XYZ
No RBX for LG040_1
Altering fields for LG040_2_SKB_XYZ
No RBX for LG040_2
Altering fields for LG040_3_SKB_XYZ
No RBX for LG040_3
Altering fields for LG040_4_SKB_XYZ
No RBX for LG040_4
Altering fields for LG040_5_SKB_XYZ
No RBX for LG040_5
Altering fields for LG040_6_SKB_XYZ
No RBX for LG040_6
Altering fields for LG661_1_SKB_XYZ
No RBX for LG661_1
Altering fields for LG661_2_SKB_XYZ
No RBX for LG661_2


0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
dtype: object

In [15]:
# Rensar rådata från onödig information

import arcpy, os, glob, shutil
from pathlib import Path
import pandas as pd

def alterfield(powerline):    
    LG = powerline[0]
    line = powerline[1]
    
    datasetnames = ["all"]
    fieldnames = [version]
    
    for dataset in datasetnames:
        print(dataset)
        for fieldname in fieldnames:
            fc_SKB = f"{dataset}_{fieldname}"
            if "all" in dataset:
                SKB_location = os.path.join(prj_dir,gdb,f"{dataset}")
            else:
                SKB_location = os.path.join(prj_dir,gdb,f"{dataset}")
            print(SKB_location)
            SKB_path = os.path.join(SKB_location, fc_SKB)
 
            # Activating workspace
            arcpy.env.workspace = SKB_path
            arcpy.env.overwriteOutput = True

            if arcpy.Exists(fc_SKB):
                SKB_XYZ_fc = fc_SKB
                print(f"Altering fields for {dataset}_{fieldname}")
                path_joined = os.path.join(SKB_location, f'{dataset}_{fieldname}')
                field_names = [f.name for f in arcpy.ListFields(SKB_path)]

                if "GIS_ID" in field_names:    
                        arcpy.management.DeleteField(
                            path_joined,
                            "GIS_ID")
                if "MAX_TILLVAXT" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "MAX_TILLVAXT")
                if "created_user" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "created_user")
                if "created_date" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "created_date")
                if "last_edited_user" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "last_edited_user")
                if "last_edited_date" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "last_edited_date")
                if "ATGARDSAR" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "ATGARDSAR")
                if "LEVERANTOR" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "LEVERANTOR")
                if "LEVERANTORS_DATUM" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "LEVERANTORS_DATUM")
                
                dataset = SKB_location
                sr = arcpy.SpatialReference("SWEREF99_TM")
                arcpy.DefineProjection_management(dataset, sr)
                # Assigning domains to fields
                arcpy.management.AssignDomainToField(
                    fc_SKB, "LEDNINGSGATA", "cvd_LEDNINGSGATA")
                arcpy.management.AssignDomainToField(
                    fc_SKB, "LITTERA", "cvd_LITTERA_LEDNING")
                arcpy.management.AssignDomainToField(
                    fc_SKB, "Insamlingsmetod", "cvd_INMATNINGSMETOD")
                arcpy.management.AssignDomainToField(
                    fc_SKB, "Matosakerhet_Plan", "cvd_MATOSAKERHET")
                arcpy.management.AssignDomainToField(
                    fc_SKB, "Matosakerhet_Hojd", "cvd_MATOSAKERHET")
    
LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(alterfield, axis=1)

all
C:\Users\semnve\Desktop\RBX_och_SKB\SKB_2023_11_01\SKB_2023_11_01.gdb\all
Altering fields for all_2023_11_01
all
C:\Users\semnve\Desktop\RBX_och_SKB\SKB_2023_11_01\SKB_2023_11_01.gdb\all
Altering fields for all_2023_11_01
all
C:\Users\semnve\Desktop\RBX_och_SKB\SKB_2023_11_01\SKB_2023_11_01.gdb\all
Altering fields for all_2023_11_01
all
C:\Users\semnve\Desktop\RBX_och_SKB\SKB_2023_11_01\SKB_2023_11_01.gdb\all
Altering fields for all_2023_11_01
all
C:\Users\semnve\Desktop\RBX_och_SKB\SKB_2023_11_01\SKB_2023_11_01.gdb\all
Altering fields for all_2023_11_01
all
C:\Users\semnve\Desktop\RBX_och_SKB\SKB_2023_11_01\SKB_2023_11_01.gdb\all
Altering fields for all_2023_11_01
all
C:\Users\semnve\Desktop\RBX_och_SKB\SKB_2023_11_01\SKB_2023_11_01.gdb\all
Altering fields for all_2023_11_01
all
C:\Users\semnve\Desktop\RBX_och_SKB\SKB_2023_11_01\SKB_2023_11_01.gdb\all
Altering fields for all_2023_11_01


0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
dtype: object

## Ta bort feature datasets
Rensar bort onödiga featureklasser från leveransdatabasen.
TODO: går det att göra en bättre "mall-databas" om sådan används, eller låta bli att alls skapa de onödiga featureklasserna?

In [16]:
import arcpy
 
# Set environment settings
arcpy.env.workspace = gdb

features = [
    "ej_akuta",
    "akuta",
    "clean_RBX_points",
    "mittfas",
    "SKB_XYmZ",
    "SKB_XYZ",
    "traffpunkter",
    "Wires",
    "Gator2021",
    "SKB_template",
    "STATIONSOMRADE"]

for feature in features:
    arcpy.management.Delete(os.path.join(gdb, feature))